# E-76 Obligated Federal Funds Data Exploration
### From the Division of Local Assistance 

E-76's or "Authorizations to Proceed" is a federal authorization of funds to establish the reimbursement date for a phase of work 

#### Data Sources: 
- https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated

In this notebook, we will be looking to see what types of data is in the spreadsheet as well as begin to clean it for futher analysis


In [82]:
##one time load 
#!pip install plotly-express
#!pip install contextily
#!pip install openpyxl
#!pip install geopandas


In [4]:
import pandas as pd
import geopandas as gpd
from siuba import _, mutate, count, filter, group_by, ungroup
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import contextily as ctx

## Reading Excel as a data dictionary
Can use this approach, but reccomend the next with Concat. 

In [2]:
#obligated = pd.read_excel('obligated-projects.xlsx', sheet_name=['2014-2018', '2019', '2020', '2021'])

In [3]:
#print(obligated)

{'2014-2018':         Location PREFIX Project NO             Agency prepared date  \
0      Obligated   BPMP  5904(121)    Humboldt County    2018-12-18   
1      Obligated     ER  32D0(008)   Mendocino County    2018-12-17   
2      Obligated     ER  4820(004)    Humboldt County    2018-12-07   
3      Obligated    CML  5924(244)  Sacramento County    2018-12-11   
4      Obligated    CML  5924(214)  Sacramento County    2018-12-05   
...          ...    ...        ...                ...           ...   
13025  Obligated  HSIPL  5211(028)           El Cajon    2013-12-05   
13026  Obligated   BHLS  5403(020)             Carson    2013-12-12   
13027  Obligated   HPLU  5958(047)           Imperial    2013-11-14   
13028  Obligated   BRLS  5166(014)           Larkspur    2013-10-09   
13029  Obligated    CML  5932(084)    Tuolumne County    2017-10-31   

      submit  to HQ date HQ review date submit to FHWA date To FMIS date  \
0             2018-12-18     2018-12-18          2018-12-

In [4]:
#print(type(obligated))

<class 'dict'>


In [5]:
#print(type(obligated['2020']))

<class 'pandas.core.frame.DataFrame'>


In [6]:
#obligated['2020'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2472 entries, 0 to 2471
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                2472 non-null   object        
 1   PREFIX                  2472 non-null   object        
 2   Project NO              2472 non-null   object        
 3   Agency                  2472 non-null   object        
 4   prepared date           2467 non-null   datetime64[ns]
 5   submit  to HQ date      2467 non-null   datetime64[ns]
 6   HQ review date          2464 non-null   datetime64[ns]
 7   submit to FHWA date     2464 non-null   datetime64[ns]
 8   To FMIS date            2464 non-null   datetime64[ns]
 9   Fed requested           2472 non-null   float64       
 10  AC requested            2472 non-null   float64       
 11  total requested         2472 non-null   float64       
 12  Status Comment          2472 non-null   object  

In [7]:
#obligated['2021'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                1941 non-null   object        
 1   PREFIX                  1941 non-null   object        
 2   Project NO              1941 non-null   object        
 3   Agency                  1941 non-null   object        
 4   prepared date           1941 non-null   datetime64[ns]
 5   submit  to HQ date      1941 non-null   datetime64[ns]
 6   HQ review date          1941 non-null   datetime64[ns]
 7   submit to FHWA date     1941 non-null   datetime64[ns]
 8   To FMIS date            1941 non-null   datetime64[ns]
 9   Fed requested           1941 non-null   float64       
 10  AC requested            1941 non-null   float64       
 11  total requested         1941 non-null   float64       
 12  Status Comment          1941 non-null   object  

In [8]:
#obligated['2019'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2578 entries, 0 to 2577
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                2578 non-null   object        
 1   PREFIX                  2578 non-null   object        
 2   Project NO              2578 non-null   object        
 3   Agency                  2578 non-null   object        
 4   prepared date           2578 non-null   datetime64[ns]
 5   submit  to HQ date      2578 non-null   datetime64[ns]
 6   HQ review date          2578 non-null   datetime64[ns]
 7   submit to FHWA date     2578 non-null   datetime64[ns]
 8   To FMIS date            2578 non-null   datetime64[ns]
 9   Fed requested           2578 non-null   float64       
 10  AC requested            2578 non-null   float64       
 11  total requested         2578 non-null   object        
 12  Status Comment          2578 non-null   object  

In [9]:
#obligated['2014-2018'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13030 entries, 0 to 13029
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                13030 non-null  object 
 1   PREFIX                  13030 non-null  object 
 2   Project NO              13030 non-null  object 
 3   Agency                  13030 non-null  object 
 4   prepared date           13025 non-null  object 
 5   submit  to HQ date      13025 non-null  object 
 6   HQ review date          13025 non-null  object 
 7   submit to FHWA date     13029 non-null  object 
 8   To FMIS date            13029 non-null  object 
 9   Fed requested           13029 non-null  float64
 10  AC requested            13030 non-null  float64
 11  total requested         13029 non-null  float64
 12  Status Comment          13030 non-null  object 
 13  Locode                  11531 non-null  float64
 14  DIST                    13030 non-null

In [10]:
#here we look to see the different data frames
#obligated['2020'].head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BRLO,5910(115),Mendocino County,2020-01-07,2020-01-07,2020-01-08,2020-01-09,2020-01-14,680000.0,...,5.0,NaN,Reynolds Highway Over Outlet Creek,Replacement Of The Existing One-lane Bailey Br...,1,2020-01-03,2020-01-07,NON-MPO,2021-01-04,NaN
1,Obligated,ACSTP,40A0(047),Lakeport,2019-12-09,2019-12-09,2020-01-10,2020-01-16,2020-01-22,0.0,...,6.0,NaN,"Hartley Street, ~300' North Of Anastasia Dr. I...","Replace Failed Culvert, Junction Box, And Head...",1,2019-11-25,2019-12-09,NON-MPO,NaN,NaN
2,Obligated,ACSTP,40A0(046),Lakeport,2019-11-25,2019-11-27,2020-01-10,2020-01-16,2020-01-22,0.0,...,6.0,NaN,"In The City Of Lakeport, South Main Street 167...",Permanent Restoration,1,2019-11-25,2019-11-27,NON-MPO,NaN,NaN
3,Obligated,ACSTP,38F0(001),Mendocino County,2019-12-12,2019-12-12,2020-01-10,2020-01-24,2020-01-28,0.0,...,4.0,NaN,"East Road, Cr 230, Redwood Valley",Replace Guard Rail And Fix Wall Destroyed By F...,1,2019-12-02,2019-12-12,NON-MPO,NaN,NaN
4,Obligated,ACSTP,40A0(043),Mendocino County,2019-12-18,2019-12-18,2020-01-10,2020-01-16,2020-01-22,0.0,...,6.0,NaN,"Mountain View Road Pm 20.00, Cr 510",Remove Residual Slide Material Install Rip Rap,1,2019-12-18,2019-12-18,NON-MPO,NaN,NaN


## Reading Excel with Concat

In [6]:
#attempt 2
## code source: https://pbpython.com/pandas-excel-tabs.html
df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=None), ignore_index=True)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:68: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,9.0,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,7.0,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,6.0,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,0.0,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,0.0,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [8]:
df.tail()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
20016,Obligated,BRLO,5955(095),Orange County,2021-08-20 00:00:00,2021-08-27 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,82000.0,...,16.0,ORA020501,"Silverado Canyon Road Over Silverado Creek, Ap...",Bridge Replacement (tc).,2,2021-08-20 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20017,Obligated,HSIPL,5073(093),Orange,2021-08-17 00:00:00,2021-08-27 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-09-08 00:00:00,21000.0,...,9.0,SCAG015,Intersection Of Glassell St. & Palmyra Ave.,Install A New 2-phase Traffic Signal,2,2021-07-28 00:00:00,2021-08-26 00:00:00,SCAG,NaT,NaN
20018,Obligated,HSIPL,5073(078),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Glassell Street And Walnut Avenue,Protected Left-turn Signal Phasing,4,2021-08-13 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20019,Obligated,HSIPL,5073(080),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Chapman Avenue And James Street,Install Protected Left Turn Signal Phasing,4,2021-08-12 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20020,Obligated,CMSTPLN,6071(129),Orange County Transportation Authority,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-31 00:00:00,2021-08-31 00:00:00,2021-09-09 00:00:00,8000000.0,...,9.0,ORA100511,State Route 55 From Interstate 405 To Intersta...,Add One Mixed-flow And High-occupancy Lane In ...,6,2021-06-01 00:00:00,2021-08-22 00:00:00,SCAG,NaT,NaN


In [9]:
print(len(df))

20021


In [10]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
13032,Obligated,BRLO,5910(112),Mendocino County,2018-11-30 00:00:00,2018-12-28 00:00:00,2019-01-08 00:00:00,2019-01-09 00:00:00,2019-01-14 00:00:00,260000.00,...,5.0,230-0000-0031,East Hill Road Over Davis Creek; Bridge No. 10...,Revegetation And Mitigation Monitoring For Bri...,1,2018-11-28 00:00:00,2018-12-12 00:00:00,NON-MPO,NaN,NaN
9842,Obligated,ER,4820(003),Humboldt County,2015-02-17,2015-02-18,2015-02-20,2015-02-20,2015-02-20,1466.00,...,0.0,NaN,"Mattole Road, Pm 43.17",Emergency Restoration,2,NaN,NaN,NON-MPO,NaN,NaN
2352,Obligated,CML,5227(058),Delano,2018-02-05,2018-02-05,2018-02-07,2018-02-16,2018-03-07,442932.00,...,19.0,KER160508,Cecil Ave. Between Albany St. And Melcher Rd.,Pave Existing Dirt Shoulder,1,2018-01-09,2018-01-29,KCOG,NaN,NaN
1131,Obligated,ER,32L0(234),Humboldt County,2018-06-27,2018-06-27,2018-06-27,2018-06-27,2018-07-03,126580.19,...,6.0,NaN,Humboldt County On Shelter Cove Road Pm 7.6,Emergency Repair,2,2018-06-27,2018-06-27,NON-MPO,NaN,NaN
12881,Obligated,STPL,5306(026),Campbell,2014-01-21,2014-01-21,2014-01-23,2014-01-23,2014-01-27,279000.00,...,4.0,SCL130003,Hamilton Ave Between Phoenix Dr. And West City...,Rehabilitate Roadway,1,NaN,NaN,MTC,NaN,NaN
16886,Obligated,ATPL,5942(275),Fresno County,2020-04-17 00:00:00,2020-07-06 00:00:00,2020-07-08 00:00:00,2020-07-10 00:00:00,2020-07-20 00:00:00,2632119.00,...,10.0,FRE170001,"Fowler Avenue, Bliss Avenue, Gonser Avenue, La...","Install Warning Lights, Warning Signs, Constru...",4,2020-06-30 00:00:00,2020-07-06 00:00:00,CFCG,NaN,NaN
12940,Obligated,CML,5157(067),Madera,2014-01-09,2014-01-09,2014-01-09,2014-01-09,2014-01-09,206283.28,...,0.0,MAD202054,Gill St. Between Pine St. And Noble St.,Const. Time-fill Cng Fueling System,4,NaN,NaN,MCTC,NaN,NaN
10622,Obligated,SRTSL,5319(014),La Palma,2014-10-22,2014-10-22,2014-10-22,2014-10-22,2014-10-23,-11793.24,...,1.0,SCAG016,Walker St.: Crescent Ave. To La Palma Ave.; Cr...,Install Vehicle Speed Radar Feedback Signs,3,NaN,NaN,SCAG,NaN,NaN
604,Obligated,STPL,5184(032),Tehachapi,2018-07-20,2018-07-27,2018-09-06,2018-09-10,2018-09-12,26559.00,...,2.0,KER180403,On Tehachapi Boulevard From Curry St. To Davis...,Rehab Ac.,1,2018-07-16,2018-07-25,KCOG,NaN,NaN
12712,Obligated,STPTCIF,6071(061),Ora Co Trans Au,2013-09-25,2013-10-02,2014-02-21,2014-02-21,2014-02-21,-7388560.00,...,0.0,ORA020826,"Tustin Avenue / Rose Drive From 1,100 Feet S/o...",New Overhead,6,NaN,NaN,SANDAG,NaN,NaN


In [11]:
df.Agency = df.Agency.str.title()

In [12]:
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)

In [13]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [14]:
df['Waiting Days'].unique()

array([nan])

In [15]:
df.drop('Waiting Days', axis=1, inplace=True)

In [16]:
df['Today'].unique()

array([nan, '2020-01-01', '2019-06-06 9:00 am', '2021-01-04',
       Timestamp('2021-10-01 00:00:00'), NaT], dtype=object)

In [17]:
df.drop('Today', axis=1, inplace=True)

In [18]:
df.drop('Unnamed: 28', axis=1, inplace=True)

In [19]:
df[["ProjectID", "ProjectNO"]] = df["Project NO"].str.split(pat="(", expand=True)


In [20]:
df.ProjectNO = [x.replace(")", "") for x in df.ProjectNO]

In [21]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,5904,121
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,244
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,214


## Locode and Project ID Columns 

Hypothesis:
    Project IDs and Locodes, or unique agency identifiers, are the same 
    
As it stands now, these columns have a mix of letters and numbers, making it hard to compare the two. 
In this block we will work to convert these columns, while trying to not convert the non-numeric numbers. 

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                20021 non-null  object 
 1   PREFIX                  20021 non-null  object 
 2   Project NO              20021 non-null  object 
 3   Agency                  20021 non-null  object 
 4   prepared date           20011 non-null  object 
 5   submit  to HQ date      20011 non-null  object 
 6   HQ review date          20008 non-null  object 
 7   submit to FHWA date     20012 non-null  object 
 8   To FMIS date            20012 non-null  object 
 9   Fed requested           20020 non-null  float64
 10  AC requested            20021 non-null  float64
 11  total requested         20020 non-null  object 
 12  Status Comment          20021 non-null  object 
 13  Locode                  18522 non-null  object 
 14  DIST                    20021 non-null

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                20021 non-null  object 
 1   PREFIX                  20021 non-null  object 
 2   Project NO              20021 non-null  object 
 3   Agency                  20021 non-null  object 
 4   prepared date           20011 non-null  object 
 5   submit  to HQ date      20011 non-null  object 
 6   HQ review date          20008 non-null  object 
 7   submit to FHWA date     20012 non-null  object 
 8   To FMIS date            20012 non-null  object 
 9   Fed requested           20020 non-null  float64
 10  AC requested            20021 non-null  float64
 11  total requested         20020 non-null  object 
 12  Status Comment          20021 non-null  object 
 13  Locode                  18522 non-null  object 
 14  DIST                    20021 non-null

In [29]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO
11575,Obligated,HSIPL,5953(697),Los Angeles County,2014-06-18,2014-06-18,2014-06-18,2014-06-27,2014-06-27,20700.00,0.00,23000.00,Authorized,5953.00,7,E-76 approved on E-76 approved on,0.00,9.00,0.00,SCAG015,Traffic Signal Improvements At Intersection Of...,Traffic Signal Improvements,1,NaN,NaN,SCAG,5953,697
2686,Obligated,ER,32L0(026),Mendocino County,2018-01-12,2018-01-12,2018-01-19,2018-01-19,2018-01-24,85438.00,-85438.00,0.00,Authorized,5910.00,1,E-76 approved on,0.00,7.00,5.00,NaN,"Bell Springs Road, Cr 324 Pm 17.70",Permanent Restoration,3,2018-01-12,2018-01-12,NON-MPO,32L0,026
14087,Obligated,STPL,5184(032),Tehachapi,2019-04-04 00:00:00,2019-04-09 00:00:00,2019-04-09 00:00:00,2019-04-16 00:00:00,2019-04-18 00:00:00,327441.00,0.00,569705,Authorized,5184,9,E-76 approved on,7.00,7.00,2.00,KER180403,On Tehachapi Boulevard From Curry St. To Hayes...,Rehab Ac. With Pavement Interlayer Mat,2,2019-04-02 00:00:00,2019-04-03 00:00:00,KCOG,5184,032
14495,Obligated,ACSTP,38Y0(016),Los Angeles County,2019-08-12 00:00:00,2019-08-12 00:00:00,2019-08-26 00:00:00,2019-08-27 00:00:00,2019-08-30 00:00:00,0.00,80215.70,90608.50,Authorized,5953,7,E-76 approved on,4.00,15.00,3.00,NaN,Mulholland Highway Over Las Virgenes Creek,"Remove And Replace Damaged Sidewalk, Wingwall ...",1,2019-08-08 00:00:00,2019-08-12 00:00:00,SCAG,38Y0,016
7373,Obligated,HRRRL,5951(150),Santa Barbara County,2016-02-05,2016-02-05,2016-02-11,2016-02-11,2016-02-23,210369.00,0.00,233744.00,Authorized,5951.00,5,E-76 approved on Prepared on 10/6/2015,0.00,6.00,12.00,LOCALHSIP,Betteravia Rd.(simas Rd.) Between Black Rd. An...,Install Rumble Strips And Flashing Beacons,3,NaN,NaN,SBCG,5951,150
15695,Obligated,BRLS,5176(011),Ross,2020-01-10 00:00:00,2020-01-10 00:00:00,2020-01-10 00:00:00,2020-01-10 00:00:00,2020-01-16 00:00:00,-16555.00,0.00,-18700.00,Authorized,5176,4,E-76 approved on,8.00,0.00,6.00,VAR110045,Norwood Ave. Bridge Over Ross Creek,Bridge Rehabilitation,2,2020-01-02 00:00:00,2020-01-03 00:00:00,MTC,5176,011
4654,Obligated,BRLO,5940(114),Mariposa County,2017-03-15,2017-03-15,2017-03-24,2017-03-24,2017-04-07,300000.00,0.00,300000.00,Authorized,5940.00,10,E-76 approved on Prepared on 2/14/2017,0.00,9.00,14.00,HBPLOCAL,Incline Road Over Moss Canyon (bridge 40c0064),Bridge Replacement (tc),2,NaN,NaN,NON-MPO,5940,114
15214,Obligated,BRLOZE,5951(118),Santa Barbara County,2019-11-08 00:00:00,2019-11-08 00:00:00,2019-11-08 00:00:00,2019-11-08 00:00:00,2019-11-22 00:00:00,-0.30,0.00,0,Authorized,5951,5,E-76 approved on,8.00,0.00,14.00,CT14,Bridge #51c0039 -0.3 Miles N Of Hwy 101 On Rin...,Install Seismic Gates #51c0039,6,2019-10-31 00:00:00,2019-11-01 00:00:00,SBCG,5951,118
15098,Obligated,ER,32L0(247),Humboldt County,2019-11-04 00:00:00,2019-11-04 00:00:00,2019-11-05 00:00:00,2019-11-05 00:00:00,2019-11-06 00:00:00,32242.62,-32242.62,0,Authorized,5904,1,E-76 approved on,3.00,1.00,1.00,NaN,"In Humboldt County, Mattole Road Pm 39.98",Permanent Restoration,3,2019-11-01 00:00:00,2019-11-01 00:00:00,NON-MPO,32L0,247
539,Obligated,HSIPL,5083(016),Redlands,2018-08-16,2018-08-21,2018-09-12,2018-09-14,2018-09-19,0.00,0.00,0.00,Authorized,5083.00,8,E-76 approved on,0.00,24.00,5.00,SCAG015,Intersection Of Pioneer Avenue And Orange Street,Install Traffic Signals And Left-turn Phasing ...,4,2018-08-16,2018-08-21,SCAG,5083,016


In [30]:
## dataset has a column NBIL, 32L0, 40A0, 5.92E+03, 38Y0, 38R0

## for now, leaving the errors. 

In [35]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

        
# https://stackoverflow.com/questions/56474599/how-to-ignore-errors-in-pandas-to-numeric-and-change-str-to-int

In [36]:
df['Locode'] = df['Locode'].apply(get_num)

In [37]:
df['ProjectID'] = df['ProjectID'].apply(get_num)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                20021 non-null  object 
 1   PREFIX                  20021 non-null  object 
 2   Project NO              20021 non-null  object 
 3   Agency                  20021 non-null  object 
 4   prepared date           20011 non-null  object 
 5   submit  to HQ date      20011 non-null  object 
 6   HQ review date          20008 non-null  object 
 7   submit to FHWA date     20012 non-null  object 
 8   To FMIS date            20012 non-null  object 
 9   Fed requested           20020 non-null  float64
 10  AC requested            20021 non-null  float64
 11  total requested         20020 non-null  object 
 12  Status Comment          20021 non-null  object 
 13  Locode                  18522 non-null  object 
 14  DIST                    20021 non-null

In [39]:
compare_col = np.where(df["Locode"] == df["ProjectID"], True, False)
df["compare_id_locode"] = compare_col
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,214,True


In [40]:
df['compare_id_locode'].value_counts()

True     16316
False     3705
Name: compare_id_locode, dtype: int64

In [41]:
group = df.groupby('Agency')
crosswalk_all = group.apply(lambda x: x['Locode'].unique())
print(crosswalk_all)

Agency
Access Services                                               [6312]
Agoura Hills                                             [5435, nan]
Alameda                                                       [5014]
Alameda - Contra Costa Transit District                       [6002]
Alameda Corridor Transportation Authority                     [6246]
                                                      ...           
Yrts                                                           [nan]
Yuba City                                                     [5163]
Yuba County                                                   [5916]
Yucaipa                                      [5457, nan, 5954, NBIL]
Yucca Valley                                             [5466, nan]
Length: 667, dtype: object


In [42]:
crosswalk_all.to_csv('crosswalk_all.csv')

In [43]:
#https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

nan_rows = df[df['Locode'].isnull()]

In [44]:
nan_rows.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
8156,Obligated,SRTSL,5954(119),San Bernardino,2015-10-22,2015-10-22,2015-10-22,2015-10-22,2015-10-28,-5548.75,0.00,47417.93,Authorized,NaN,8,E-76 approved on E-76 approved on,0.00,0.00,6.00,SCAG016,East Side Of Almond Avenue From North Of Upas ...,"Road Rehabilitation, Curb, Gutter, And Sidewalk",4,NaN,NaN,COFCG,5954,119,False
11003,Obligated,CMLN,6054(058),Riv Co Trans Co,2014-07-24,2014-08-07,2014-08-14,2014-08-14,2014-08-21,5542731.00,0.00,6260851.00,Authorized,NaN,8,E-76 approved on E-76 approved on,14.00,7.00,7.00,NaN,State Route 91 From Adams Street To The 60/91/...,Hov Lanes,2,NaN,NaN,STANCOG,6054,058,False
9775,Obligated,CML,5165(069),Turlock,2015-02-18,2015-02-23,2015-02-25,2015-03-02,2015-03-04,-6204.85,0.00,12023.79,Authorized,NaN,10,E-76 approved on E-76 approved on,5.00,7.00,2.00,VA03,South Of Taylor Rd. - Near Crowell Rd To End O...,Construct Bike/ped Path,2,NaN,NaN,SCAG,5165,069,False
9496,Obligated,RPSTPL,5184(022),Tehachapi,2015-04-03,2015-04-03,2015-04-03,2015-04-03,2015-04-08,0.00,0.00,0.00,Authorized,NaN,9,E-76 approved on E-76 approved on,0.00,0.00,5.00,KER121009,South Side Of Tehachapi Boulevard From Snyder ...,"Sidewalk, Ada Curb Ramps, Landscaping, & Lighting",3,NaN,NaN,SCAG,5184,022,False
8892,Obligated,RPSTPL,6066(093),Sd Assoc Gov'T,2015-06-19,2015-06-19,2015-06-19,2015-06-19,2015-06-24,-49539.68,0.00,0.00,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,0.00,5.00,SAN21,W Of I-5 From H St In Chula Vista In North To ...,Bikeway,2,NaN,NaN,SCAG,6066,093,False
10441,Obligated,CML,5244(025),Patterson,2014-11-17,2014-11-17,2014-11-25,2014-11-25,2014-11-25,-31377.18,0.00,-34677.40,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,8.00,0.00,STANCOG,Intersection N. First St. And Walnut Ave.,Signal Installation,4,NaN,NaN,SCAG,5244,025,False
10890,Obligated,BRLS,5107(002),Santa Monica,2014-08-14,2014-08-19,2014-08-25,2014-09-03,2014-09-05,15550332.00,3629730.00,22584082.00,Authorized,NaN,7,E-76 approved on E-76 approved on,5.00,15.00,2.00,LA000800 &,California Incline: Btwn Ocean Ave. And Pch ...,Replace. Of Bluffs Bridge # 53c0543 (tc),6,NaN,NaN,SJCOG,5107,002,False
11387,Obligated,HSIPL,5378(033),Palmdale,2014-07-08,2014-07-08,2014-07-09,2014-07-10,2014-07-12,-500.00,0.00,0.00,Authorized,NaN,7,E-76 approved on E-76 approved on,0.00,2.00,2.00,SCAG015,Ave. R And 55th Street East,"Traffic Signal, Curb Ramps, Shoulder",3,NaN,NaN,SJCOG,5378,033,False
11880,Obligated,ER,24M0(001),Inyo,2014-04-18,2014-04-18,2014-05-19,2014-05-19,2014-05-19,100000.00,0.00,100000.00,Authorized,NaN,9,E-76 approved on E-76 approved on,0.00,31.00,0.00,NaN,On Trona Wildrose Rd From Panament Valley Rd. ...,Emergency Opening,1,NaN,NaN,NON-MPO,24M0,001,False
8948,Obligated,CML,5006(680),Los Angeles,2015-06-03,2015-06-03,2015-06-03,2015-06-16,2015-06-17,-44316.35,0.00,0.00,Authorized,NaN,7,E-76 approved on E-76 approved on,0.00,13.00,1.00,LAF1522,Various Locations In The City Of Los Angeles (...,Upgrading Existing Roadway Grates,4,NaN,NaN,SCAG,5006,680,False


In [45]:
print(len(nan_rows))

1499


In [43]:
#checking to see if these NaN values are also in the downloadable data. 

In [32]:
# There are the same number of NaNs (1499) on the excel sheet. 
# So without these, there are still 2,206 

## Converting remaining columns 

In [46]:
print(df['prepared date'].dtypes)

object


In [47]:
df['prepared date'] = pd.to_datetime(df['prepared date'])
print(df['prepared date'].dtypes)

datetime64[ns]


In [48]:
df['To FMIS date'] = pd.to_datetime(df['To FMIS date'])
print(df['To FMIS date'].dtypes)

datetime64[ns]


In [49]:
df['submit to FHWA date'] = pd.to_datetime(df['submit to FHWA date'])
print(df['submit to FHWA date'].dtypes)

datetime64[ns]


In [50]:
df['submit  to HQ date'] = pd.to_datetime(df['submit  to HQ date'])
print(df['submit  to HQ date'].dtypes)

datetime64[ns]


In [51]:
df['HQ review date'] = pd.to_datetime(df['HQ review date'])
print(df['HQ review date'].dtypes)

datetime64[ns]


In [52]:
df['fmisyear'] = df['To FMIS date'].dt.year

In [53]:
df['fmisyear'] = df['fmisyear'].astype(float).fillna(0)
df['fmisyear'] = df['fmisyear'].astype('int')
df['fmisyear'] = df['fmisyear'].astype('str')

In [54]:
df['Date Request Initiated'] = pd.to_datetime(df['Date Request Initiated'])
print(df['Date Request Initiated'].dtypes)

datetime64[ns]


In [55]:
df['date completed request'] = pd.to_datetime(df['date completed request'])
print(df['date completed request'].dtypes)

datetime64[ns]


In [56]:
df['completedyear'] = df['date completed request'].dt.year

In [57]:
df['completedyear'] = df['completedyear'].astype(float).fillna(0)
df['completedyear'] = df['completedyear'].astype('int')
df['completedyear'] = df['completedyear'].astype('str')

In [58]:
compare_col2 = np.where(df["fmisyear"] == df["completedyear"], True, False)
df["compare_start_end"] = compare_col2
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode,fmisyear,completedyear,compare_start_end
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,121,True,2018,0,False
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008,False,2018,2018,True
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004,False,2018,2018,True
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,244,True,2018,2018,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,214,True,2018,2018,True


In [59]:
df['compare_start_end'].value_counts()

False    11461
True      8560
Name: compare_start_end, dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                20021 non-null  object        
 1   PREFIX                  20021 non-null  object        
 2   Project NO              20021 non-null  object        
 3   Agency                  20021 non-null  object        
 4   prepared date           20011 non-null  datetime64[ns]
 5   submit  to HQ date      20011 non-null  datetime64[ns]
 6   HQ review date          20008 non-null  datetime64[ns]
 7   submit to FHWA date     20012 non-null  datetime64[ns]
 8   To FMIS date            20012 non-null  datetime64[ns]
 9   Fed requested           20020 non-null  float64       
 10  AC requested            20021 non-null  float64       
 11  total requested         20020 non-null  object        
 12  Status Comment          20021 non-null  object

In [61]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode,fmisyear,completedyear,compare_start_end
6244,Obligated,HSIPL,5946(163),Tulare County,2016-07-18,2016-07-18,2016-07-18,2016-07-21,2016-07-26,117000.00,0.00,130000.00,Authorized,5946,6,E-76 approved on Prepared on 10/6/2015,0.00,3.00,5.00,TUL12-144,Avenue 232 Between Road 36 And Road 76 (palm S...,Install Left-turn Lanes And Edgeline Rumble St...,1,NaT,NaT,TCAG,5946,163,True,2016,0,False
9591,Obligated,STPL,5123(015),Sebastopol,2015-02-24,2015-02-24,2015-03-16,2015-03-20,2015-03-23,249392.00,0.00,249392.00,Authorized,5123,4,E-76 approved on E-76 approved on,0.00,24.00,3.00,SON130004,Keating Avenue From North Main Street To Pitt ...,Rehabilitate Roadway (tc),1,NaT,NaT,MTC,5123,015,True,2015,0,False
2205,Obligated,HSIPL,5354(035),Union City,2018-03-16,2018-03-16,2018-03-16,2018-03-16,2018-03-21,-0.40,0.00,0.00,Authorized,5354,4,E-76 approved on,NaN,0.00,5.00,VAR110007,Alverado-niles Road: Between Decoto Road To Ma...,Pedestrian Safety Improvments,5,NaT,NaT,MTC,5354,035,True,2018,0,False
15648,Obligated,STPL,5002(173),Sacramento,2019-12-11,2019-12-16,2019-12-31,2020-01-06,2020-01-10,0.00,5496496.00,6690496.00,Authorized,5002,3,E-76 approved on,20.00,21.00,4.00,SAC24742,Meadowview Road From Freeport Blvd. To Light R...,Streetscape Project,2,2019-11-26,2019-12-13,SACOG,5002,173,True,2020,2019,False
8967,Obligated,CML,5169(040),El Centro,2015-06-12,2015-06-12,2015-06-15,2015-06-15,2015-06-16,442149.00,0.00,719556.00,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,3.00,1.00,IMP120619,Buena Vista Between 6th/8th St.,"Construct Sidewalk, C&g,ada Ramps",2,NaT,NaT,SCAG,5169,040,False,2015,0,False
15993,Obligated,CMLNI,5258(037),Chowchilla,2020-01-08,2020-01-08,2020-01-08,2020-02-06,2020-02-11,-807.35,0.00,-549.41,Authorized,5258,6,E-76 approved on,6.00,29.00,5.00,MAD302056,City Of Chowchilla,Purchase One Cng Street Sweeper,2,2020-01-02,2020-01-03,MCTC,5258,037,True,2020,2020,True
5863,Obligated,STPL,5916(112),Yuba County,2016-07-26,2016-08-25,2016-08-29,2016-09-02,2016-09-13,717000.00,0.00,992305.00,Authorized,5916,3,E-76 approved on Prepared on 9/29/2016,30.00,8.00,11.00,YUB16042,Intersection Of Olivehurst Ave. And Powerline Rd.,"Construct Roundabout, Widen Sidewalks To Facil...",1,NaT,NaT,SACOG,5916,112,True,2016,0,False
13060,Obligated,ER,32L0(319),Humboldt County,2019-01-10,2019-01-10,2019-02-22,2019-03-01,2019-03-06,1823.71,0.00,2060,Authorized,5904,1,E-76 approved on,1.00,50.00,5.00,NaN,In Humboldt County On Mattole Road Pm 5.10,Permanent Restoration,1,2019-01-09,2019-01-09,NON-MPO,32L0,319,False,2019,2019,True
976,Obligated,STPL,5008(172),Stockton,2018-06-29,2018-07-02,2018-07-10,2018-07-10,2018-07-19,5508954.00,0.00,5561754.00,Authorized,5008,10,E-76 approved on,18.00,8.00,9.00,SJ07-3071,"In Stockton, Various Locations: 1) Miner Avenu...",Pavement Resurfacing (tc),1,2018-06-14,2018-06-28,SJCG,5008,172,True,2018,2018,True
1796,Obligated,BRLO,5907(032),Lassen County,2018-04-09,2018-04-09,2018-04-24,2018-04-24,2018-04-27,-184799.23,0.00,-177530.23,Authorized,5907,2,E-76 approved on,7.00,15.00,3.00,HBPLOCAL,Susan River Overfl Br #5 On Mapes Ln (br. #07c...,Bridge Replacement (tc),8,2018-04-02,2018-04-09,NON-MPO,5907,032,True,2018,2018,True


In [62]:
from siuba.data import cars_sql
from siuba import group_by, mutate, show_query

In [63]:
df[_.compare_start_end == False]

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode,fmisyear,completedyear,compare_start_end
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,121,True,2018,0,False
9,Obligated,STPL,5060(259),Fresno,2018-10-29,2018-10-29,2018-12-21,2018-12-27,2018-12-27,-16774.40,0.00,10919.60,Authorized,5060,6,E-76 approved on,NaN,59.00,0.00,FRE110622,Eaton Trail 1/4 Mile North Of Audubon Drive To...,Trail Resurfacing And Bridge Repair (tc),4,NaT,NaT,CFCG,5060,259,True,2018,0,False
13,Obligated,HPLULN,6071(043),Orange County Transportation Authority,2018-12-19,2018-12-19,2018-12-21,2018-12-21,2018-12-27,100450000.00,0.00,0.00,Authorized,6071,12,E-76 approved on,NaN,2.00,6.00,NaN,I-405 From The Los Angeles County Line To Sr 73,Add One Mixed-flow Lane In Each Direction,4,NaT,NaT,SCAG,6071,043,True,2018,0,False
15,Obligated,BRLO,5917(082),Nevada County,2018-10-24,2018-10-24,2018-12-20,2018-12-20,2018-12-23,-139683.27,0.00,-134704.97,Authorized,5917,3,E-76 approved on,NaN,57.00,3.00,HBP_NEV,Garden Bar Rd At Little Wolf Creek #2; ~4.3 Mi...,Bridge Replacement (tc),5,NaT,NaT,NON-MPO,5917,082,True,2018,0,False
18,Obligated,BRLO,5910(086),Mendocino County,2018-12-10,2018-12-10,2018-12-20,2018-12-20,2018-12-21,-0.01,0.00,-0.42,Authorized,5910,1,E-76 approved on,NaN,10.00,1.00,HBPLOCAL,"Reeves Canyon Road At Forsythe Creek, Bridge N...","Bridge Replacement, Added Capacity (tc)",6,NaT,NaT,NON-MPO,5910,086,True,2018,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19823,FTA Transferred,FTACML,6262(021),San Joaquin Regional Rail Commission,2021-06-30,2021-06-30,2021-06-30,2021-07-27,2021-08-03,7500000.00,0.00,7500000.00,Prog Code Z0E3,6262,10,FTA transferred on 8/3/2021,NaN,27.00,7.00,NaN,NaN,FTA Transfer,1,NaT,NaT,SJCG,6262,021,True,2021,0,False
19824,FTA Transferred,FTACML,6187(008),Southern California Regional Rail Authority,2021-07-26,2021-07-26,2021-07-26,2021-08-03,2021-08-03,25000000.00,0.00,25000000.00,Prog Code Z40E,6187,7,FTA transferred on 8/3/2021,NaN,8.00,0.00,NaN,NaN,FTA Transfer,1,NaT,NaT,SCAG,6187,008,True,2021,0,False
19825,FTA Transferred,FTACML,6261(025),Victor Valley Transit Authority,2021-07-19,2021-07-19,2021-07-19,2021-08-02,2021-08-03,5115000.00,0.00,5115000.00,Prog Code Z400,6261,8,FTA transferred on 8/3/2021,NaN,14.00,1.00,NaN,NaN,FTA Transfer,1,NaT,NaT,SCAG,6261,025,True,2021,0,False
19826,FTA Transferred,FTACML,6262(021),San Joaquin Regional Rail Commission,2021-06-30,2021-06-30,2021-06-30,2021-07-27,2021-08-03,7500000.00,0.00,7500000.00,Prog Code Z0E3,6262,10,FTA transferred on 8/3/2021,NaN,27.00,7.00,NaN,NaN,FTA Transfer,1,NaT,NaT,SJCG,6262,021,True,2021,0,False


In [64]:
#intersting that some of the rows that show a False between the FMIS date and the completed date,
## are not showing any dates for when the request was iniatated...

## Summarizing Data
What are the values in these columns

In [65]:
df['fmisyear'].value_counts()

2018    2840
2014    2747
2015    2587
2019    2578
2016    2486
2017    2349
2020    2235
2021    2170
2013      20
0          9
Name: fmisyear, dtype: int64

In [66]:
df['Status Comment'].value_counts()

Authorized        19470
Prog Code Z400      147
Prog Code M400       73
Prog Code Z230       60
Prog Code Z003       40
Prog Code M40E       33
Prog Code M0E3       22
Prog Code M23E       17
Prog Code RPS9       14
Prog Code Z40E       14
Prog Code M230       14
Prog Code Z301       14
Prog Code M240       13
Prog Code L23E       10
Prog Code L40E        9
Prog Code Z240        9
Prog Code M003        8
Prog Code L230        6
Prog Code Z23E        6
prog code Z230        3
Prog Code L240        3
Prog Code L400        3
Prog Code L23R        3
Prog Code Z24E        3
Prog Code Q230        2
prog code Z240        2
Prog Code H230        2
Prog Code 33C0        2
Prog Code Z0E3        2
Prog Code M950        2
Prog Code Z300        2
Prog Code M301        2
Prog Code Z950        1
Prog Code M95E        1
Prog Code L24E        1
Prog Code RPS0        1
Prog Code M3E1        1
prog code Z24E        1
prog code             1
prog code Z400        1
Prog Code M30E        1
Prog Code Z233  

In [67]:
df['Location'].value_counts()

Obligated          19470
FTA Transferred      543
DISTRICT               5
HQ                     3
Name: Location, dtype: int64

In [68]:
df['Type of Work'].nunique()

6768

In [69]:
df['PREFIX'].nunique()

388

In [70]:
prefix = df['PREFIX'].value_counts()

In [74]:
prefix.head()

CML      3290
HSIPL    3245
STPL     2763
ER       1568
BRLO     1390
Name: PREFIX, dtype: int64

In [72]:
print((prefix == 1).sum())

113


In [80]:
prefix.sort_index(inplace=True)
prefix

6140000      2
ACNHPI       2
ACST-ER    233
ACSTP      303
ASCTP        1
          ... 
VPPL        19
VPPP         3
VPPPL        5
VPPTCS       1
WZDXL20      1
Name: PREFIX, Length: 388, dtype: int64

In [81]:
#Getting a CSV to create a detailed crosswalk with defintions. 
prefix.to_csv('count_grant_types.csv')

In [84]:
df.to_csv('clean.csv')

## Convert to CSV for future analysis 